#### **Rivaldo Lumelino, Alexandr Voronovich**
#### **CSC 36000 – Final Project**

#### **Problem 1**
##### **How can agents communicate and coordinate when network is poor (urban congestion or rural coverage gaps)?**
##### **Without solution → collisions, inefficiency, unsafe behavior**

In [1]:
import random

In [ ]:
import math

In [2]:
from collections import deque

##### **Agent Class**

In [ ]:
class Agent:
    def __init__(self, agent_id, x, y):
        self.id = agent_id # save agent id
        self.x = x # x coordinate
        self.y = y # y coordinate
        self.vx = 0 # velosity
        self.vy = 0 # velosity